In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None, 10])

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
#noisylabels10 = mnist.test.labels
#print(labels)
def shuffle_rows(arr,rows):
    np.random.shuffle(arr[rows[0]:rows[1]])

#shuffle_rows(noisylabels10,[0,1000])

<b>2 CNN Begins here</b>

In [4]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [5]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

In [6]:
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

x_image = tf.reshape(x, [-1, 28, 28, 1])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

In [7]:
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [8]:
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

In [9]:
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [10]:
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

In [11]:
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

arrTrain = np.ndarray((22,1), dtype = float)
arrTest = np.ndarray((22,1), dtype = float)
count = 0
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(2200):
        batch = mnist.train.next_batch(50)
        shuffle_rows(batch[1],[0,5])
        if i % 100 == 0:
            train_accuracy = accuracy.eval(feed_dict={
                x: batch[0], y_: batch[1], keep_prob: 1.0})
            arrTrain[count] = train_accuracy
            count+=1
            print('step %d, training accuracy %g' % (i, train_accuracy))
        train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
    count = 0 
    for i in range(2200):
        batch = mnist.test.next_batch(50)
        #shuffle_rows(batch[1],[0,5])
        if i % 100 == 0:
            test_accuracy = accuracy.eval(feed_dict={x: batch[0], y_: batch[1], keep_prob: 1.0})
            print('test accuracy %g' % test_accuracy)
            arrTest[count] = test_accuracy
            count+=1

step 0, training accuracy 0.04
step 100, training accuracy 0.74
step 200, training accuracy 0.84
step 300, training accuracy 0.9
step 400, training accuracy 0.82
step 500, training accuracy 0.88
step 600, training accuracy 0.78
step 700, training accuracy 0.92
step 800, training accuracy 0.92
step 900, training accuracy 0.88
step 1000, training accuracy 0.9
step 1100, training accuracy 0.84
step 1200, training accuracy 0.88
step 1300, training accuracy 0.92
step 1400, training accuracy 0.84
step 1500, training accuracy 0.82
step 1600, training accuracy 0.78
step 1700, training accuracy 0.88
step 1800, training accuracy 0.84
step 1900, training accuracy 0.78
step 2000, training accuracy 0.84
step 2100, training accuracy 0.76
test accuracy 0.94
test accuracy 0.96
test accuracy 0.92
test accuracy 1
test accuracy 0.96
test accuracy 0.94
test accuracy 0.98
test accuracy 1
test accuracy 0.96
test accuracy 0.94
test accuracy 0.98
test accuracy 0.96
test accuracy 0.98
test accuracy 0.98
test a

<b>This is for visualizing the performance while training this model</b>

In [12]:
import matplotlib.pyplot as plt
plt.plot(arrTrain)
plt.yticks(np.arange(0.0, 1.2, 0.1))
plt.title('Evaluating training accuracy with 10% Noise')
plt.xticks(np.arange(1,22))
plt.ylabel('Accuracy')
plt.xlabel('Iterations')
plt.show()

KeyboardInterrupt: 

<b>This is for visualizing the performance of this model on testing data</b>

In [ ]:
import matplotlib.pyplot as plt
plt.plot(arrTest)
plt.yticks(np.arange(0.0, 1.2, 0.1))
plt.title('Evaluating testing accuracy with 10% Noise')
plt.xticks(np.arange(1,22))
plt.ylabel('Accuracy')
plt.xlabel('Iterations')
plt.show()